# Analisis Perilaku Pelanggan Cafe dalam Promo Bulanan

In [1]:
# Import libraries that perhaps required

import os
import sys
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, nltk, warnings
import matplotlib.cm as cm
import itertools
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
from wordcloud import WordCloud, STOPWORDS
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from IPython.display import display, HTML
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
%matplotlib inline

In [2]:
datapath = os.getcwd()
datapath

'D:\\Data Satria\\Python Notebook Learn\\November 2024\\Cafe+Rewards+Offers'

Check out the current directory to see the list of files available.

In [3]:
import glob
cwd = os.getcwd()
listdir = os.listdir()
listdir

['.ipynb_checkpoints',
 'CafeCustAnalysis.ipynb',
 'customers.csv',
 'data_dictionary.csv',
 'events.csv',
 'offers.csv']

In [66]:
#Load all the files into dataframes

#Assign the datafile path
csv_files = glob.glob(os.path.join(datapath, "*.csv"))
display(csv_files)

#Create the bucket for the dfs
dfs = {}

#Assign the dataframe
for file in csv_files:
    # Get the filename as the dict key
    file_name = os.path.basename(file).replace(".csv", "")
    # Read the data
    df = pd.read_csv(file)
    # Assign the data into the bucket(dict)
    dfs[file_name] = df
    
df_cust = dfs['customers']
df_datadict = dfs['data_dictionary']
df_events = dfs['events']
df_offers = dfs['offers']

['D:\\Data Satria\\Python Notebook Learn\\November 2024\\Cafe+Rewards+Offers\\customers.csv',
 'D:\\Data Satria\\Python Notebook Learn\\November 2024\\Cafe+Rewards+Offers\\data_dictionary.csv',
 'D:\\Data Satria\\Python Notebook Learn\\November 2024\\Cafe+Rewards+Offers\\events.csv',
 'D:\\Data Satria\\Python Notebook Learn\\November 2024\\Cafe+Rewards+Offers\\offers.csv']

In [67]:
# Now I'll try to handle the cust data first.
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       17000 non-null  object 
 1   became_member_on  17000 non-null  int64  
 2   gender            14825 non-null  object 
 3   age               17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


As we can see on the brief info above, seems if a lot of members didn't input their gender and income at once. I'll imput it as O for Others and 0 Respectively.

In [69]:
#df_cust = df_cust.dropna(subset=['gender'])
df_cust['gender'] = df_cust['gender'].fillna('O')
df_cust['income'] = df_cust['income'].fillna(0)
display(df_cust.info())
df_cust.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       17000 non-null  object 
 1   became_member_on  17000 non-null  int64  
 2   gender            17000 non-null  object 
 3   age               17000 non-null  int64  
 4   income            17000 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


None

,customer_id,became_member_on,gender,age,income
0,68be06ca386d4c31939f3a4f0e3dd783,20170212,O,118,0.0
1,0610b486422d4921ae7d2bf64640c50b,20170715,F,55,112000.0
2,38fe809add3b4fcf9315a9694bb96ff5,20180712,O,118,0.0
3,78afa995795e4d85b5d9ceeca43f5fef,20170509,F,75,100000.0
4,a03223e636434f42ac4c3df47e8bac43,20170804,O,118,0.0


Let's see the basic info of the other data.

In [7]:
display(df_datadict.info())
display(df_events.info())
display(df_offers.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Table        18 non-null     object
 1   Field        15 non-null     object
 2   Description  18 non-null     object
dtypes: object(3)
memory usage: 564.0+ bytes


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  306534 non-null  object
 1   event        306534 non-null  object
 2   value        306534 non-null  object
 3   time         306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   offer_id    10 non-null     object
 1   offer_type  10 non-null     object
 2   difficulty  10 non-null     int64 
 3   reward      10 non-null     int64 
 4   duration    10 non-null     int64 
 5   channels    10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


None

Let's see the data dictionary first.

In [8]:
df_datadictT= df_datadict.T
df_datadictT
display(df_datadict)

,Table,Field,Description
0,offers,NaN,Details on the offers sent to customers during...
1,offers,offer_id,Unique offer ID (primary key)
2,offers,offer_type,"type of offer: bogo (buy one, get one), discou..."
3,offers,difficulty,minimum amount required to spend in order to b...
4,offers,reward,reward (in dollars) obtained by completing the...
5,offers,duration,days a customer has to complete the offer once...
6,offers,channels,list of marketing channels used to send the of...
7,customers,NaN,Demographic data for each member
8,customers,customer_id,Unique customer ID (primary key)
9,customers,became_member_on,Date when the customer created their account (...


In [9]:
df_events.head(5)

,customer_id,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [10]:
# Unique values of event column
unique_values1 = df_events['event'].unique()
distinct_count1 = df_events['event'].nunique()
display(unique_values1)
display(f"Number of distinct values: {distinct_count1}")

# Unique values of value column
unique_values2 = df_events['value'].unique()
distinct_count2 = df_events['value'].nunique()
display(unique_values2)
display(f"Number of distinct values: {distinct_count2}")

# Unique values of time column
unique_values3 = df_events['time'].unique()
distinct_count3 = df_events['time'].nunique()
display(unique_values3)
display(f"Number of distinct values: {distinct_count3}")



array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

'Number of distinct values: 4'

array(["{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}",
       "{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}",
       "{'offer id': '2906b810c7d4411798c6938adc9daaa5'}", ...,
       "{'amount': 685.07}", "{'amount': 405.04}", "{'amount': 476.33}"],
      dtype=object)

'Number of distinct values: 5121'

array([  0,   6,  12,  18,  24,  30,  36,  42,  48,  54,  60,  66,  72,
        78,  84,  90,  96, 102, 108, 114, 120, 126, 132, 138, 144, 150,
       156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228,
       234, 240, 246, 252, 258, 264, 270, 276, 282, 288, 294, 300, 306,
       312, 318, 324, 330, 336, 342, 348, 354, 360, 366, 372, 378, 384,
       390, 396, 402, 408, 414, 420, 426, 432, 438, 444, 450, 456, 462,
       468, 474, 480, 486, 492, 498, 504, 510, 516, 522, 528, 534, 540,
       546, 552, 558, 564, 570, 576, 582, 588, 594, 600, 606, 612, 618,
       624, 630, 636, 642, 648, 654, 660, 666, 672, 678, 684, 690, 696,
       702, 708, 714])

'Number of distinct values: 120'

In [11]:
df_events['time_in_day'] = np.ceil(df_events['time'] / 24).astype(int)
df_events.tail(5)


,customer_id,event,value,time,time_in_day
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714,30
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714,30
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714,30
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714,30
306533,c02b10e8752c4d8e9b73f918558531f7,transaction,{'amount': 4.05},714,30


In [12]:
import ast

# Convert string representation of dictionaries to actual dictionaries and try to extract its value
df_events['value'] = df_events['value'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_events['value'] = df_events['value'].apply(lambda x: {} if pd.isna(x) else x)
expanded_df = df_events['value'].apply(pd.Series)

# Rename the time column 
df_events = df_events.rename(columns={'time':'time_in_hour'})
expanded_df

,offer id,amount,offer_id,reward
0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
2,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
3,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
4,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN
...,...,...,...,...
306529,NaN,1.59,NaN,NaN
306530,NaN,9.53,NaN,NaN
306531,NaN,3.61,NaN,NaN
306532,NaN,3.53,NaN,NaN


Let's merge it

In [15]:
df_events = pd.concat([df_events.drop(columns=['value']), expanded_df], axis=1)
df_events.head(2)
display(df_events.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   customer_id   306534 non-null  object 
 1   event         306534 non-null  object 
 2   time_in_hour  306534 non-null  int64  
 3   time_in_day   306534 non-null  int64  
 4   offer id      134002 non-null  object 
 5   amount        138953 non-null  float64
 6   offer_id      33579 non-null   object 
 7   reward        33579 non-null   float64
dtypes: float64(2), int64(2), object(4)
memory usage: 18.7+ MB


None

In [16]:
# First, rename the offer column to its actual context based on data dictionary.
df_events = df_events.rename(columns={'offer id':'offer_received'})

# Check out all the NaN Value
nan_check = df_events[df_events['offer_received'].notnull() & df_events['offer_id'].notnull() & df_events['reward'].notnull()]
display(nan_check)

nan_check2 = df_events[df_events['offer_received'].notnull()]
display(nan_check2)

nan_check3 = df_events[df_events['offer_id'].notnull() & df_events['reward'].notnull()]
display(nan_check3)

,customer_id,event,time_in_hour,time_in_day,offer_received,amount,offer_id,reward


,customer_id,event,time_in_hour,time_in_day,offer_received,amount,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
306441,d56386cf344c4829bbf420d1895dca37,offer viewed,714,30,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,NaN
306450,9b51e8797290403b90d09d864dec4b94,offer viewed,714,30,3f207df678b143eea3cee63160fa8bed,NaN,NaN,NaN
306483,84fb57a7fe8045a8bf6236738ee73a0f,offer viewed,714,30,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,NaN
306490,abc4359eb34e4e2ca2349da2ddf771b6,offer viewed,714,30,3f207df678b143eea3cee63160fa8bed,NaN,NaN,NaN


,customer_id,event,time_in_hour,time_in_day,offer_received,amount,offer_id,reward
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,0,0,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,2.0
12672,fe97aa22dd3e48c8b143116a8403dd52,offer completed,0,0,NaN,NaN,fafdcd668e3743c1bb461111dcafc2a4,2.0
12679,629fc02d56414d91bca360decdfa9288,offer completed,0,0,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0
12692,676506bad68e4161b9bbaffeb039626b,offer completed,0,0,NaN,NaN,ae264e3637204a6fb9bb56bc8210ddfd,10.0
12697,8f7dd3b2afe14c078eb4f6e6fe4ba97d,offer completed,0,0,NaN,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0
...,...,...,...,...,...,...,...,...
306475,0c027f5f34dd4b9eba0a25785c611273,offer completed,714,30,NaN,NaN,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer completed,714,30,NaN,NaN,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0
306506,b895c57e8cd047a8872ce02aa54759d6,offer completed,714,30,NaN,NaN,fafdcd668e3743c1bb461111dcafc2a4,2.0
306509,8431c16f8e1d440880db371a68f82dd0,offer completed,714,30,NaN,NaN,fafdcd668e3743c1bb461111dcafc2a4,2.0


As we can see on the data preview above, that there are customers who only received the offer and not doing any transactions. <br>
Let's split it into different data.

In [37]:
# Make a dataframe for every
df_offrec = df_events[df_events['offer_received'].notnull()]
df_offrec = df_offrec.dropna(axis=1)
df_offrec = df_offrec.rename(columns={'offer_received':'offer_id'})
df_offrec.reset_index(drop=True, inplace=True)
df_offrec

,customer_id,event,time_in_hour,time_in_day,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,0,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0
...,...,...,...,...,...
133997,d56386cf344c4829bbf420d1895dca37,offer viewed,714,30,5a8bc65990b245e5a138643cd4eb9837
133998,9b51e8797290403b90d09d864dec4b94,offer viewed,714,30,3f207df678b143eea3cee63160fa8bed
133999,84fb57a7fe8045a8bf6236738ee73a0f,offer viewed,714,30,5a8bc65990b245e5a138643cd4eb9837
134000,abc4359eb34e4e2ca2349da2ddf771b6,offer viewed,714,30,3f207df678b143eea3cee63160fa8bed


In [28]:
# Crosscheck the offers table
display(df_offers.isna().sum())
display(df_offers.info())
df_offers.head(10)

offer_id      0
offer_type    0
difficulty    0
reward        0
duration      0
channels      0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   offer_id    10 non-null     object
 1   offer_type  10 non-null     object
 2   difficulty  10 non-null     int64 
 3   reward      10 non-null     int64 
 4   duration    10 non-null     int64 
 5   channels    10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


None

,offer_id,offer_type,difficulty,reward,duration,channels
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,10,7,"['email', 'mobile', 'social']"
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,10,5,"['web', 'email', 'mobile', 'social']"
2,3f207df678b143eea3cee63160fa8bed,informational,0,0,4,"['web', 'email', 'mobile']"
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,5,7,"['web', 'email', 'mobile']"
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20,5,10,"['web', 'email']"
5,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,7,3,7,"['web', 'email', 'mobile', 'social']"
6,fafdcd668e3743c1bb461111dcafc2a4,discount,10,2,10,"['web', 'email', 'mobile', 'social']"
7,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,3,"['email', 'mobile', 'social']"
8,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,"['web', 'email', 'mobile', 'social']"
9,2906b810c7d4411798c6938adc9daaa5,discount,10,2,7,"['web', 'email', 'mobile']"


In [29]:
# Make another df for offers reward.

df_reward = df_events[df_events['offer_id'].notnull() & df_events['reward'].notnull()]
df_reward = df_reward.dropna(axis=1)
df_reward.reset_index(drop=True, inplace=True)
df_reward

,customer_id,event,time_in_hour,time_in_day,offer_id,reward
0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,0,0,2906b810c7d4411798c6938adc9daaa5,2.0
1,fe97aa22dd3e48c8b143116a8403dd52,offer completed,0,0,fafdcd668e3743c1bb461111dcafc2a4,2.0
2,629fc02d56414d91bca360decdfa9288,offer completed,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0
3,676506bad68e4161b9bbaffeb039626b,offer completed,0,0,ae264e3637204a6fb9bb56bc8210ddfd,10.0
4,8f7dd3b2afe14c078eb4f6e6fe4ba97d,offer completed,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0
...,...,...,...,...,...,...
33574,0c027f5f34dd4b9eba0a25785c611273,offer completed,714,30,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0
33575,a6f84f4e976f44508c358cc9aba6d2b3,offer completed,714,30,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0
33576,b895c57e8cd047a8872ce02aa54759d6,offer completed,714,30,fafdcd668e3743c1bb461111dcafc2a4,2.0
33577,8431c16f8e1d440880db371a68f82dd0,offer completed,714,30,fafdcd668e3743c1bb461111dcafc2a4,2.0


In [30]:
# Make another df for normal buyer transaction.

df_normalts = df_events[df_events['amount'].notnull()]
df_normalts = df_normalts.dropna(axis=1)
df_normalts.reset_index(drop=True, inplace=True)
df_normalts

,customer_id,event,time_in_hour,time_in_day,amount
0,02c083884c7d45b39cc68e1314fec56c,transaction,0,0,0.83
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,0,34.56
2,54890f68699049c2a04d415abc25e717,transaction,0,0,13.23
3,b2f1cd155b864803ad8334cdf13c4bd2,transaction,0,0,19.51
4,fe97aa22dd3e48c8b143116a8403dd52,transaction,0,0,18.97
...,...,...,...,...,...
138948,b3a1272bc9904337b331bf348c3e8c17,transaction,714,30,1.59
138949,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,30,9.53
138950,a00058cf10334a308c68e7631c529907,transaction,714,30,3.61
138951,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,30,3.53


In [35]:
df_allevents = df_events.sort_values(by=['time_in_day','customer_id'], ascending=[True, True])
df_allevents

,customer_id,event,time_in_hour,time_in_day,offer_received,amount,offer_id,reward
3066,0011e0d4e6b944f998e987f904e8c1e5,offer received,0,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN,NaN
1889,0020c2b971eb4e9188eac86d93036a77,offer received,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
2652,003d66b6608740288d6cc97a6903f4f0,offer received,0,0,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,NaN
8354,00426fe3ffde4c6b9cb9ad6d077a13ea,offer received,0,0,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,NaN
9960,005500a7188546ff8a767329a2f7c76a,offer received,0,0,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
304925,ffac285a64714aca8efd8d61a5606e84,transaction,708,30,NaN,7.77,NaN,NaN
304926,ffac285a64714aca8efd8d61a5606e84,offer completed,708,30,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0
305865,ffc77312ccda49fca19506f1cdf44923,transaction,714,30,NaN,1.65,NaN,NaN
305307,ffc8a38e0972421785b00f80c92c8085,transaction,708,30,NaN,36.07,NaN,NaN


Now let's just join the offer table.

In [45]:
new_dfevt = pd.merge(df_offrec, df_offers, on='offer_id', how='inner')
df_sortedoffrec = new_dfevt.sort_values(by=['offer_type', 'customer_id', 'event', 'time_in_hour'], ascending=[True,True,True,True])
df_sortedoffrec

,customer_id,event,time_in_hour,time_in_day,offer_id,offer_type,difficulty,reward,duration,channels
69645,0009655768c64bdeb2e877511632db8f,offer received,408,17,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,"['web', 'email', 'mobile', 'social']"
87743,0009655768c64bdeb2e877511632db8f,offer viewed,456,19,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,"['web', 'email', 'mobile', 'social']"
25894,00116118485d4dfda04fdbaba9a87b5c,offer received,168,7,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,"['web', 'email', 'mobile', 'social']"
114160,00116118485d4dfda04fdbaba9a87b5c,offer received,576,24,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,"['web', 'email', 'mobile', 'social']"
43173,00116118485d4dfda04fdbaba9a87b5c,offer viewed,216,9,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,"['web', 'email', 'mobile', 'social']"
...,...,...,...,...,...,...,...,...,...,...
93103,fff3ba4757bd42088c044ca26d73817a,offer received,504,21,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,3,"['email', 'mobile', 'social']"
109177,fff3ba4757bd42088c044ca26d73817a,offer viewed,540,23,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,3,"['email', 'mobile', 'social']"
90471,fff8957ea8b240a6b5e634b6ee8eafcf,offer received,504,21,3f207df678b143eea3cee63160fa8bed,informational,0,0,4,"['web', 'email', 'mobile']"
22777,fffad4f4828548d1b5583907f2e9906b,offer received,168,7,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,3,"['email', 'mobile', 'social']"


In [60]:
# Calculate ratio description for each offer receiver and viewed
eventratio = df_sortedoffrec.groupby('event').size().reset_index(name='sum of each event')
total_sum = eventratio['sum of each event'].sum()
display(total_sum)

diff_count = eventratio[eventratio['event'] == 'offer received']['sum of each event'].values[0]
eventratio['difference'] = diff_count - eventratio['sum of each event']
eventratio['percentage'] = (eventratio['sum of each event'] / total_sum) * 100

eventratio



np.int64(134002)

,event,sum of each event,difference,percentage
0,offer received,76277,0,56.922285
1,offer viewed,57725,18552,43.077715


As we can see on the preview table above, customers tend to only receive the offer and not viewed it. With the difference aroud 18552 offers.

In [70]:
# Check out all of the dataframe that have been created

display(df_cust)
display(df_offrec)
display(df_reward)
display(df_normalts) # Normalts means Normal Transaction
display(df_allevents)
display(df_sortedoffrec)

,customer_id,became_member_on,gender,age,income
0,68be06ca386d4c31939f3a4f0e3dd783,20170212,O,118,0.0
1,0610b486422d4921ae7d2bf64640c50b,20170715,F,55,112000.0
2,38fe809add3b4fcf9315a9694bb96ff5,20180712,O,118,0.0
3,78afa995795e4d85b5d9ceeca43f5fef,20170509,F,75,100000.0
4,a03223e636434f42ac4c3df47e8bac43,20170804,O,118,0.0
...,...,...,...,...,...
16995,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,F,45,54000.0
16996,2cb4f97358b841b9a9773a7aa05a9d77,20180713,M,61,72000.0
16997,01d26f638c274aa0b965d24cefe3183f,20170126,M,49,73000.0
16998,9dc1421481194dcd9400aec7c9ae6366,20160307,F,83,50000.0


,customer_id,event,time_in_hour,time_in_day,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,0,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0
...,...,...,...,...,...
133997,d56386cf344c4829bbf420d1895dca37,offer viewed,714,30,5a8bc65990b245e5a138643cd4eb9837
133998,9b51e8797290403b90d09d864dec4b94,offer viewed,714,30,3f207df678b143eea3cee63160fa8bed
133999,84fb57a7fe8045a8bf6236738ee73a0f,offer viewed,714,30,5a8bc65990b245e5a138643cd4eb9837
134000,abc4359eb34e4e2ca2349da2ddf771b6,offer viewed,714,30,3f207df678b143eea3cee63160fa8bed


,customer_id,event,time_in_hour,time_in_day,offer_id,reward
0,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,0,0,2906b810c7d4411798c6938adc9daaa5,2.0
1,fe97aa22dd3e48c8b143116a8403dd52,offer completed,0,0,fafdcd668e3743c1bb461111dcafc2a4,2.0
2,629fc02d56414d91bca360decdfa9288,offer completed,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0
3,676506bad68e4161b9bbaffeb039626b,offer completed,0,0,ae264e3637204a6fb9bb56bc8210ddfd,10.0
4,8f7dd3b2afe14c078eb4f6e6fe4ba97d,offer completed,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0
...,...,...,...,...,...,...
33574,0c027f5f34dd4b9eba0a25785c611273,offer completed,714,30,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0
33575,a6f84f4e976f44508c358cc9aba6d2b3,offer completed,714,30,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0
33576,b895c57e8cd047a8872ce02aa54759d6,offer completed,714,30,fafdcd668e3743c1bb461111dcafc2a4,2.0
33577,8431c16f8e1d440880db371a68f82dd0,offer completed,714,30,fafdcd668e3743c1bb461111dcafc2a4,2.0


,customer_id,event,time_in_hour,time_in_day,amount
0,02c083884c7d45b39cc68e1314fec56c,transaction,0,0,0.83
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,0,34.56
2,54890f68699049c2a04d415abc25e717,transaction,0,0,13.23
3,b2f1cd155b864803ad8334cdf13c4bd2,transaction,0,0,19.51
4,fe97aa22dd3e48c8b143116a8403dd52,transaction,0,0,18.97
...,...,...,...,...,...
138948,b3a1272bc9904337b331bf348c3e8c17,transaction,714,30,1.59
138949,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,30,9.53
138950,a00058cf10334a308c68e7631c529907,transaction,714,30,3.61
138951,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,30,3.53


,customer_id,event,time_in_hour,time_in_day,offer_received,amount,offer_id,reward
3066,0011e0d4e6b944f998e987f904e8c1e5,offer received,0,0,3f207df678b143eea3cee63160fa8bed,NaN,NaN,NaN
1889,0020c2b971eb4e9188eac86d93036a77,offer received,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
2652,003d66b6608740288d6cc97a6903f4f0,offer received,0,0,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,NaN
8354,00426fe3ffde4c6b9cb9ad6d077a13ea,offer received,0,0,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,NaN
9960,005500a7188546ff8a767329a2f7c76a,offer received,0,0,ae264e3637204a6fb9bb56bc8210ddfd,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
304925,ffac285a64714aca8efd8d61a5606e84,transaction,708,30,NaN,7.77,NaN,NaN
304926,ffac285a64714aca8efd8d61a5606e84,offer completed,708,30,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0
305865,ffc77312ccda49fca19506f1cdf44923,transaction,714,30,NaN,1.65,NaN,NaN
305307,ffc8a38e0972421785b00f80c92c8085,transaction,708,30,NaN,36.07,NaN,NaN


,customer_id,event,time_in_hour,time_in_day,offer_id,offer_type,difficulty,reward,duration,channels
69645,0009655768c64bdeb2e877511632db8f,offer received,408,17,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,"['web', 'email', 'mobile', 'social']"
87743,0009655768c64bdeb2e877511632db8f,offer viewed,456,19,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,"['web', 'email', 'mobile', 'social']"
25894,00116118485d4dfda04fdbaba9a87b5c,offer received,168,7,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,"['web', 'email', 'mobile', 'social']"
114160,00116118485d4dfda04fdbaba9a87b5c,offer received,576,24,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,"['web', 'email', 'mobile', 'social']"
43173,00116118485d4dfda04fdbaba9a87b5c,offer viewed,216,9,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,"['web', 'email', 'mobile', 'social']"
...,...,...,...,...,...,...,...,...,...,...
93103,fff3ba4757bd42088c044ca26d73817a,offer received,504,21,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,3,"['email', 'mobile', 'social']"
109177,fff3ba4757bd42088c044ca26d73817a,offer viewed,540,23,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,3,"['email', 'mobile', 'social']"
90471,fff8957ea8b240a6b5e634b6ee8eafcf,offer received,504,21,3f207df678b143eea3cee63160fa8bed,informational,0,0,4,"['web', 'email', 'mobile']"
22777,fffad4f4828548d1b5583907f2e9906b,offer received,168,7,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,3,"['email', 'mobile', 'social']"


# The data is ready to use.

In [71]:
df_cust.to_csv('Customers_Data.csv', index=False)
df_sortedoffrec.to_csv('Sorted_Offers.csv', index=False)
df_reward.to_csv('Reward_Gained.csv', index=False)
df_normalts.to_csv('Normal_Transactions.csv', index=False)
df_allevents.to_csv('All_EventTS.csv', index=False)